## Exp Record Editting

In [11]:
import pandas as pd
import numpy as np
df = pd.read_excel(r"S:\ExpRecord_tmp.xlsx")

In [14]:
import os
os.environ['COMPUTERNAME']

'PONCELAB-OFF6'

In [10]:
!dir S:\

 Volume in drive S is Network Mapping
 Volume Serial Number is 92B3-CACF

 Directory of S:\

2023-11-06  04:33 PM    <DIR>          .
2023-11-06  04:33 PM    <DIR>          ..
2021-05-19  01:56 AM        46,959,292 Alfa-16112020-001_formatted.mat
2022-12-12  02:26 PM            36,448 Beto-30112020-002-errorMess.mat
2022-12-12  02:28 PM            52,728 Beto-31052022-002-errorMess.mat
2022-05-31  04:29 PM    <DIR>          Data-Behavior (BHV2)
2023-11-06  04:26 PM    <DIR>          Data-Ephys-MAT
2023-11-06  04:02 PM    <DIR>          Data-Ephys-Raw
2023-01-19  12:53 PM             3,162 Evol_BigGAN_AB_loadingfail.xlsx
2023-01-30  06:50 PM           364,998 Evol_BigGAN_singular_ExpsMeta.mat
2023-11-06  04:31 PM             5,467 ExpRecord_out.xlsx
2020-10-09  04:56 PM            17,958 ExpRecord_out_Alfa.xlsx
2020-10-09  05:57 PM            17,527 ExpRecord_out_Beto.xlsx
2020-04-12  11:32 PM            12,167 ExpRecord_tmp.xlsx
2019-12-09  05:00 PM           110,592 ExpSpecTable.xls
2

In [12]:
df

,ephysFN,expControlFN,stimuli,comments,Exp_collection,Expi,pref_chan,stim_size
0,-,-,-,Plan is to run,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,rf mapping (10 min),NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Evolution (20-30 min),NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Occlusion (10 min),NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Selectivity (3 min),NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,Reconstruction (20 min),NaN,NaN,NaN,NaN
6,Caos-06112023-001.pl2,231106_Caos_rfMapper_basic.bhv2,N:\Stimuli\RFs\2023\11-06-Caos-01,Rf mapping at 1:15,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,200 ml water in tank,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,Stopped after 6 block,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,5 min rf mapping,NaN,NaN,NaN,NaN


In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel(r"D:\ExpRecord_tmp.xlsx")
df_formal = pd.read_excel(r"S:\ExpSpecTable_Augment.xlsx")

In [6]:
def process_concat_cells(df, out_excel, Animal):
    """Process the raw form excel copied from onenote to well formed excel
    Filter the array using `Animal` label""
    """
    if isinstance(df,str):
        df = pd.read_excel(df)
    df = df.dropna(axis=0, how='all') # drop lines full with nan
    df = df.reset_index(drop=True)  # (index=range(df.shape[0]))  # make the index contiguous!
    row_num = df.shape[0]
    #%%
    msk = ~ df.ephysFN.isna()  # 500 rows
    # df.ephysFN[msk].str.contains("Alfa")  # 436 rows containing Alfa
    if Animal is "Alfa":
        search_str = "Alfa|ALfa"
    elif Animal is "Beto":
        search_str = "Beto"
    elif Animal is "Caos":
        search_str = "Caos"
    elif Animal is "Both":
        search_str = "Beto|Alfa|ALfa"
    else:
        search_str = "Beto|Alfa|ALfa|Caos"
    ExpEphysNames = df.ephysFN[df.ephysFN.str.contains(search_str)==True]
    print(ExpEphysNames)
    RowidEphs = ExpEphysNames.index
    ExpBhv2Names = df.expControlFN[df.expControlFN.str.contains(search_str)==True]
    RowidBhv = ExpEphysNames.index
    assert RowidEphs is RowidBhv
    #%%
    df.comments.fillna(value="", inplace=True)
    df.comments = df.comments.astype("str")
    df.stimuli.fillna(value="", inplace=True)
    df.stimuli = df.stimuli.astype("str")
    #%%
    for Expi, rowi in enumerate(RowidEphs):
        if Expi != len(RowidEphs) - 1:
            nextrow = RowidEphs[Expi + 1]
        else:
            nextrow = row_num
        print("\nExp %d\t %s\t %s"%( Expi, df.ephysFN[rowi], df.expControlFN[rowi]))
        print(df.comments[rowi:nextrow].str.cat(sep="\n"))
    # 
    stimuli_miss_cnt = 0  # Count how many stimuli entries are missed
    df_sort = df[df.ephysFN.str.contains(search_str)==True]
    df_sort = df_sort.reset_index(drop=True)
    for Expi, rowi in enumerate(RowidEphs):
        if Expi != len(RowidEphs) - 1:
            nextrow = RowidEphs[Expi + 1]
        else:
            nextrow = row_num
        df_sort.comments[Expi] = df.comments[rowi:nextrow].str.cat(sep="\n")
        df_sort.ephysFN[Expi] = df.ephysFN[rowi]
        df_sort.expControlFN[Expi] = df.expControlFN[rowi]
        if "Stimuli" in df.stimuli[rowi]:
            df_sort.stimuli[Expi] = df.stimuli[rowi]
        else:
            df_sort.stimuli[Expi] = ""
            stimuli_miss_cnt += 1
            # print out info for further examination
            print("\nExp %d\t %s\t %s" % (Expi, df.ephysFN[rowi], df.expControlFN[rowi]))
            print(df.stimuli[rowi:nextrow].str.cat(sep=""))
            if ("Abort" in df_sort.comments[Expi]) or ("abort" in df_sort.comments[Expi]):
                print("Do aborted! No worry.")
    print(stimuli_miss_cnt, "stimuli missing")
    #%%
    df_sort.to_excel(out_excel,index=False)
    return df_sort

In [37]:
(df_formal.expControlFN==name).nonzero()[0][0]

141

In [111]:
def concat_table(df_old, df_new, addexplabel=None, out_path=None):
    if isinstance(df_old,str):
        out_path = df_old
        df_old = pd.read_excel(df_old)
    if isinstance(df_new,str):
        df_new = pd.read_excel(df_new)
    # Check if the experiments in the new datatable has been recorded
    break_flag = False
    for name in df_new.expControlFN:
        if (df_old.expControlFN==name).any():
            print("%s  has been recorded in the excel index %d, please check"%(name, (df_old.expControlFN==name).nonzero()[0][0]))
            break_flag = True
    if break_flag:
        raise ValueError
    if addexplabel is not None:
        df_new.Exp_collection[:] = addexplabel
    df_cat = pd.concat([df_old,df_new], axis=0, ignore_index=True)
    df_cat.to_excel(out_path,index=False)
    return df_cat

def sort_merge_table(df_sort, addexplabel=None):
    Animal_strs = ["Alfa", "Beto"]
    df_paths = [r"S:\Exp_Record_Alfa.xlsx", r"S:\ExpSpecTable_Augment.xlsx"]
    if isinstance(df_sort,str):
        df_sort = pd.read_excel(df_sort)
    # loop through animal name and sort corresponding exp to the collection
    for animal, out_path in zip(Animal_strs, df_paths):
        print("Sort out exp for %s, adding "%animal)
        df_old = pd.read_excel(out_path) # load the old exp collection
        id_col = []
        for idx in df_sort.index:
            name = df_sort.expControlFN[idx]
            if name is np.nan:
                print("%s Empty bhv file entry encountered" % df_sort.ephysFN[idx])
                continue
            if animal.lower() in name.lower():
                if (df_old.expControlFN==name).any():
                    print("%s  has been recorded in the excel index %d, please check. Skipping."%(name, (df_old.expControlFN==name).nonzero()[0][0]))
                else:
                    id_col.append(idx)
        if len(id_col) == 0:
            print("\nNo new experiments to add! Continue.")
            continue
        df_ftr = df_sort.iloc[id_col].copy()
        print(df_ftr.expControlFN)
        if addexplabel is not None:
            df_ftr.Exp_collection[:] = addexplabel
        df_cat = pd.concat([df_old, df_ftr], axis=0, ignore_index=True)
        df_cat.to_excel(out_path,index=False) # write to the excel of all old experiments 
    return
# pd.concat([df_formal,df_sort], axis=0, ignore_index=True)

In [13]:
Animal = "Caos"#"Beto" # "Alfa" "ALfa"
df_sort = process_concat_cells(r"S:\ExpRecord_tmp.xlsx", "S:\ExpRecord_out.xlsx", Animal=Animal)

6     Caos-06112023-001.pl2
11        Caos-06112023-002
16        Caos-06112023-003
21        Caos-06112023-004
29        Caos-06112023-005
32        Caos-06112023-006
35        Caos-06112023-007
40        Caos-06112023-008
45        Caos-06112023-009
50        Caos-06112023-010
Name: ephysFN, dtype: object

Exp 0	 Caos-06112023-001.pl2	 231106_Caos_rfMapper_basic.bhv2
Rf mapping at 1:15
200 ml water in tank
Stopped after 6 block
5 min rf mapping
COMPLITED

Exp 1	 Caos-06112023-002	 231106_Caos_rfMapper_basic(1).bhv2
RF mapping at 1:27
Finished at 1:33
After 4 block
1 degree stimuli
COMPLITED

Exp 2	 Caos-06112023-003	 231106_Caos_generate_integrated.bhv2
Evolution started at 1:39 PM
trying 71 (0 0 ) 3 1 CMAES
Sparse PSTH, but let's see
MU 3-4/5
TERMINATED – Not improve in PSTH

Exp 3	 Caos-06112023-004	 231106_Caos_generate_integrated(1).bhv2
Evolution started at 1:46PM
trying 71 (0 0 ) 3 2 CMAES
Evolving from Hash
added 200 ml AJ to 300 levee (400 ml added in sum)
Stopped at 25 block

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A valu

In [63]:
Animal = "Both"#"Beto" # "Alfa" "ALfa"
df_sort = process_concat_cells(r"D:\ExpRecord_tmp.xlsx", "D:\ExpRecord_out.xlsx", Animal=Animal)


Exp 0	 Alfa64chan-19032020-001	 200319_Alfa_rfMapper_basic
001 Rf mapper 2 degrees, 2 [-8 8]
Start at 144. 5 blocks complete, stop at 149
Completed

Exp 1	 Alfa64chan-19032020-002	 200319_Alfa_rfMapper_basic(1)
002 Rf mapper, 1 degree [-2 2]
Start at 1:50. 6 blocks complete stop at 152
Completed

Exp 2	 Alfa64chan-19032020-003	 200319_Alfa_generate_integrated
003 generate integrated starts 13:59
50 (-1.0,-2.5) 4 1 ZOHA full
50 (-1.0,-2.5) 4 1 ZOHA reduced
a beautiful brown convexity
Good gap
46 mins
Completed

Exp 3	 Alfa64chan-19032020-004	 200319_Alfa_generate_integrated(2)
004 starts 14:50
GI(1) is discarded… didn't start recording
50 (-1.0,-2.5) 4 1 ZOHA reduced
50 (-1.0,-2.5) 4 1 ZOHA reduced
Try 2 reduced to see the variability 
At least one of them can potentially reach as high.
Still a brown convexity!
Note thread 1 is pretty unstable….Step size problem? Overshoot
PSTH looks promising. 
Plan to reach 40 and terminates.
Completed

Exp 4	 Alfa64chan-19032020-005	 200319_Alfa_gen

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

0 stimuli missing


In [2]:
Animal = input("Which animal to parse?")

Which animal to parse?Alfa


In [3]:
len(Animal)

4

In [64]:
df_sort

,ephysFN,expControlFN,stimuli,comments,Exp_collection,Expi,pref_chan,stim_size
0,Alfa64chan-19032020-001,200319_Alfa_rfMapper_basic,N:\Stimuli\2019-06-RF-mapping\2020-03-19-Alfa,"001 Rf mapper 2 degrees, 2 [-8 8]\nStart at 14...",NaN,NaN,NaN,NaN
1,Alfa64chan-19032020-002,200319_Alfa_rfMapper_basic(1),N:\Stimuli\2019-06-RF-mapping\2020-03-19-Alfa,"002 Rf mapper, 1 degree [-2 2]\nStart at 1:50....",NaN,NaN,NaN,NaN
2,Alfa64chan-19032020-003,200319_Alfa_generate_integrated,N:\Stimuli\2019-12-Evolutions\2020-03-19-Alfa-...,003 generate integrated starts 13:59\n50 (-1.0...,NaN,NaN,NaN,NaN
3,Alfa64chan-19032020-004,200319_Alfa_generate_integrated(2),N:\Stimuli\2019-12-Evolutions\2020-03-19-Alfa-...,004 starts 14:50\nGI(1) is discarded… didn't s...,NaN,NaN,NaN,NaN
4,Alfa64chan-19032020-005,200319_Alfa_generate_integrated(3),N:\Stimuli\2019-12-Evolutions\2020-03-19-Alfa-...,"46, SU, V1\nfull vs reduced\n46 (0,0) 3 3 zoha...",NaN,NaN,NaN,NaN
5,Beto-19032020-001,200319_Beto_rfMapper_basic,N:\Stimuli\2019-06-RF-mapping\2020-03-19-Beto,001 RFMapping 13:\n-8:8:8 \nCarlos' huge image...,NaN,NaN,NaN,NaN
6,Beto-19032020-002,200319_Beto_rfMapper_basic(1),N:\Stimuli\2019-06-RF-mapping\2020-03-19-Beto,"002 RFMapping starts 13:36\n-8:2:8\nMid image,...",NaN,NaN,NaN,NaN
7,Beto-19032020-003,200319_Beto_rfMapper_basic(2),N:\Stimuli\2019-06-RF-mapping\2020-03-19-Beto,003 RFMapping starts 13:49\n-2:1:2\nCompleted,NaN,NaN,NaN,NaN
8,Beto-19032020-004,200319_Beto_generate_integrated,N:\Stimuli\2019-12-Evolutions\2020-03-19-Beto-...,004 Generate Integrated starts 14:06\n25 [-1 -...,NaN,NaN,NaN,NaN
9,Beto-19032020-005,200319_Beto_generate_integrated(1),N:\Stimuli\2019-12-Evolutions\2020-03-19-Beto-...,005 Generate Integrated starts 15:07\n25 [-1 -...,NaN,NaN,NaN,NaN


In [66]:
sort_merge_table(df_sort, addexplabel="ReducDimen_Evol")

Sort out exp for Alfa, adding 
0            200319_Alfa_rfMapper_basic
1         200319_Alfa_rfMapper_basic(1)
2       200319_Alfa_generate_integrated
3    200319_Alfa_generate_integrated(2)
4    200319_Alfa_generate_integrated(3)
Name: expControlFN, dtype: object


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



Sort out exp for Beto, adding 
5             200319_Beto_rfMapper_basic
6          200319_Beto_rfMapper_basic(1)
7          200319_Beto_rfMapper_basic(2)
8        200319_Beto_generate_integrated
9     200319_Beto_generate_integrated(1)
10    200319_Beto_generate_integrated(2)
Name: expControlFN, dtype: object


In [12]:
if Animal == "Beto":
    table_old = r"S:\ExpSpecTable_Augment.xlsx"
elif Animal == "Alfa":
    table_old = r"S:\Exp_Record_Alfa.xlsx"
df_cat = concat_table(table_old, r"D:\ExpRecord_out.xlsx", addexplabel="ReducDimen_Evol") # "Optim_tuning"

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [22]:
df_sort.Exp_collection[:] = "SUHash"

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
!S:\ExpSpecTable_Augment.xlsx

In [30]:
!S:\Exp_Record_Alfa.xlsx

The process cannot access the file because it is being used by another process.


## Find Corresponding BHV2

In [89]:
import pandas as pd
import numpy as np
df = pd.read_excel(r"D:\ExpRecord_out.xlsx")

In [73]:
search_str='Alfa'
ExpEphysNames = df.ephysFN[df.ephysFN.str.contains(search_str)==True]
RowidEphs = ExpEphysNames.index
ExpBhv2Names = df.expControlFN[df.expControlFN.str.contains(search_str)==True]
RowidBhv = ExpBhv2Names.index

In [74]:
from os.path import pathsep,split

In [21]:
# https://www.journaldev.com/23365/python-string-to-datetime-strptime
from datetime import datetime
datetime_str = '2020-02-04-13-38-53'
date_object = datetime.strptime(datetime_str, '%Y-%m-%d-%H-%M-%S')

In [33]:
import glob
from os.path import join
import os

In [75]:
#alfa_gen_fns1 = glob.glob(join("N:/Data-Behavior (BHV2)/", "*_ALfa_generate*.bhv2"))
alfa_gen_fns = glob.glob(join("N:/Data-Behavior (BHV2)/", "*_Alfa_generate*.bhv2"))
alfa_RF_fns = glob.glob(join("N:/Data-Behavior (BHV2)/", "*_Alfa_rfMapper*.bhv2"))
#alfa_gen_fns = sorted(alfa_gen_fns1+alfa_gen_fns2)

In [76]:
len(alfa_RF_fns)

313

In [41]:
datetime_str = '2020-02-04-13-38-53'
date_obj = datetime.strptime(datetime_str, '%Y-%m-%d-%H-%M-%S')
"%s_Alfa_generate" % date_obj.strftime('%y%m%d')

'200204_Alfa_generate'

In [39]:
alfa_gen_mtime = []
for fn in alfa_gen_fns:
    TS = os.path.getmtime(fn)
    alfa_gen_mtime.append(datetime.fromtimestamp(TS))

In [71]:
for idx, stim_path in zip(RowidEphs, df.stimuli[RowidEphs]):
    if stim_path is np.nan:
        continue
    net_path, folder = split(stim_path)
    if "Evolutions" in net_path:
        print(idx, folder, df.ephysFN[idx])
        datetime_str = folder
        date_obj = datetime.strptime(datetime_str, '%Y-%m-%d-%H-%M-%S')
        bhv_tmplate = "%s" % date_obj.strftime('%y%m%d')
        candidate_fns = [fn for fn in alfa_gen_fns if bhv_tmplate in fn]
        for fn in candidate_fns:
            TS = os.path.getmtime(fn)
            Tobj = datetime.fromtimestamp(TS)
            if Tobj < date_obj:
                continue
            SZ = os.path.getsize(fn)
            print(split(fn)[1], Tobj.strftime('%H:%M:%S'), "%.1f Mb" % (SZ/(2**20)))
        print('')

0 2020-02-04-13-38-53 Alfa64chan-04022020-002
200204_ALfa_generate_integrated.bhv2 14:01:35 22.9 Mb
200204_ALfa_generate_integrated(1).bhv2 14:57:44 592.6 Mb

1 2020-02-04-14-03-42 Alfa64chan-04022020-003
200204_ALfa_generate_integrated(1).bhv2 14:57:44 592.6 Mb

3 2020-02-05-12-18-42 Alfa64chan-05022020-005
200205_ALfa_generate_integrated(3).bhv2 13:16:40 46.6 Mb
200205_ALfa_generate_integrated(4).bhv2 13:55:07 37.4 Mb
200205_ALfa_generate_integrated(2).bhv2 12:43:04 26.0 Mb

4 2020-02-05-13-18-55 Alfa64chan-05022020-007
200205_ALfa_generate_integrated(4).bhv2 13:55:07 37.4 Mb

5 2020-02-10-16-42-15 Alfa64chan-10022020-004
200210_Alfa_generate_integrated(1).bhv2 17:38:20 27.7 Mb
200210_Alfa_generate_integrated(2).bhv2 18:03:23 48.9 Mb
200210_Alfa_generate_integrated.bhv2 17:34:27 75.4 Mb

6 2020-02-11-10-30-50 Alfa64chan-11022020-003
200211_Alfa_generate_integrated.bhv2 11:40:24 48.4 Mb

9 2020-02-13-12-41-34 Alfa64chan-13022020-003
200213_Alfa_generate_integrated(2).bhv2 14:50:59 101

In [86]:
df.ephysFN[idx][11:19]

'13022020'

In [88]:
for idx, ctrlFN in zip(RowidEphs, df.expControlFN[RowidEphs]):
    if ctrlFN is np.nan:
        print(idx, df.ephysFN[idx])
        date_obj = datetime.strptime(df.ephysFN[idx][11:19], '%d%m%Y')
        bhv_tmplate = "%s" % date_obj.strftime('%y%m%d')
        candidate_fns = [fn for fn in alfa_RF_fns if bhv_tmplate in fn]
        for fn in candidate_fns:
            TS = os.path.getmtime(fn)
            Tobj = datetime.fromtimestamp(TS)
            if Tobj < date_obj:
                continue
            SZ = os.path.getsize(fn)
            print(split(fn)[1], Tobj.strftime('%H:%M:%S'), "%.1f Mb" % (SZ/(2**20)))
        print('')

2 Alfa64chan-05022020-004
200205_ALfa_rfMapper_basic.bhv2 11:59:11 21.2 Mb
200205_ALfa_rfMapper_basic(2).bhv2 12:07:42 22.6 Mb

8 Alfa64chan-13022020-002
200213_Alfa_rfMapper_basic(1).bhv2 12:06:10 26.7 Mb

10 Alfa64chan-17022020-001
200217_Alfa_rfMapper_basic(1).bhv2 12:58:37 31.9 Mb
200217_Alfa_rfMapper_basic.bhv2 12:43:13 21.5 Mb

11 Alfa64chan-17022020-002
200217_Alfa_rfMapper_basic(1).bhv2 12:58:37 31.9 Mb
200217_Alfa_rfMapper_basic.bhv2 12:43:13 21.5 Mb

12 Alfa64chan-17022020-003
200217_Alfa_rfMapper_basic(1).bhv2 12:58:37 31.9 Mb
200217_Alfa_rfMapper_basic.bhv2 12:43:13 21.5 Mb

13 Alfa64chan-17022020-004
200217_Alfa_rfMapper_basic(1).bhv2 12:58:37 31.9 Mb
200217_Alfa_rfMapper_basic.bhv2 12:43:13 21.5 Mb

15 Alfa64chan-18022020-001
200218_Alfa_rfMapper_basic.bhv2 12:14:21 39.0 Mb
200218_Alfa_rfMapper_basic(1).bhv2 12:26:22 27.0 Mb

16 Alfa64chan-18022020-002
200218_Alfa_rfMapper_basic.bhv2 12:14:21 39.0 Mb
200218_Alfa_rfMapper_basic(1).bhv2 12:26:22 27.0 Mb

18 Alfa64chan-19022

In [94]:
df

,ephysFN,expControlFN,stimuli,comments,Exp_collection,Expi,pref_chan,stim_size
0,Alfa64chan-04022020-002,200204_ALfa_generate_integrated,Stimuli\2019-12-Evolutions\2020-02-04-Alfa-01\...,"Start: 1:38 pm, generate_integrated, parallel ...",NaN,NaN,NaN,NaN
1,Alfa64chan-04022020-003,200204_ALfa_generate_integrated(1),Stimuli\2019-12-Evolutions\2020-02-04-Alfa-02\...,"Start: 2:03 pm, restarted evolution\ngenerate_...",NaN,NaN,NaN,NaN
2,Alfa64chan-05022020-005,200205_ALfa_generate_integrated(2),Stimuli\2019-12-Evolutions\2020-02-05-Alfa-01\...,005 generate_integrated SU vs Hash\nCh. 33 [...,NaN,NaN,NaN,NaN
3,Alfa64chan-05022020-007,200205_ALfa_generate_integrated(4),Stimuli\2019-12-Evolutions\2020-02-05-Alfa-03\...,007 generate_integrated SU vs Hash\nCh. 38 [0...,NaN,NaN,NaN,NaN
4,Alfa64chan-10022020-004,200210_Alfa_generate_integrated,Stimuli\2019-12-Evolutions\2020-02-10-Alfa-01\...,Ch. 7 [0 -2] 3 1 'CMAES'\nCh. 7 [0 -2] 3 2 'CM...,NaN,NaN,NaN,NaN
5,Alfa64chan-11022020-003,200211_Alfa_generate_integrated,Stimuli\2019-12-Evolutions\2020-02-11-Alfa-01\...,"SU v Hash ch. 63 [0, -2] SU 1/5 very visually ...",NaN,NaN,NaN,NaN
6,Alfa64chan-13022020-001,200213_Alfa_rfMapper_basic(1),N:\Stimuli\2019-06-RF-mapping\2020-02-13-Alfa,001RFMapping \nstarts 11:59 AM\n-8:2:8 2deg im...,NaN,NaN,NaN,NaN
7,Alfa64chan-13022020-002,200213_Alfa_generate_integrated(1),NaN,"002 generate_integrated\n63 [0, -2] 3 1 CMAES...",NaN,NaN,NaN,NaN
8,Alfa64chan-13022020-003,200213_Alfa_generate_integrated(2),Stimuli\2019-12-Evolutions\2020-02-13-Alfa-01\...,"003 generate_integrated\nRe open ML2, change t...",NaN,NaN,NaN,NaN
9,Alfa64chan-17022020-001,200217_Alfa_rfMapper_basic,NaN,001 RFMapper starts\n-1.5:0.5:1.5 mapping 1 deg,NaN,NaN,NaN,NaN


In [99]:
df.expControlFN[1].contains('Alfa')

AttributeError: 'str' object has no attribute 'contains'

In [113]:
sort_merge_table("D:/ExpRecord_out.xlsx", addexplabel="SUHash")

Sort out exp for Alfa, adding 
0        200204_ALfa_generate_integrated
1     200204_ALfa_generate_integrated(1)
2     200205_ALfa_generate_integrated(2)
3     200205_ALfa_generate_integrated(4)
4        200210_Alfa_generate_integrated
5        200211_Alfa_generate_integrated
6          200213_Alfa_rfMapper_basic(1)
7     200213_Alfa_generate_integrated(1)
8     200213_Alfa_generate_integrated(2)
9             200217_Alfa_rfMapper_basic
10         200217_Alfa_rfMapper_basic(1)
11    200217_Alfa_generate_integrated(4)
12            200218_Alfa_rfMapper_basic
13         200218_Alfa_rfMapper_basic(1)
14       200218_Alfa_generate_integrated
15            200219_Alfa_rfMapper_basic
16         200219_Alfa_rfMapper_basic(1)
17    200219_Alfa_generate_integrated(2)
18            200220_Alfa_rfMapper_basic
19         200220_Alfa_rfMapper_basic(1)
20       200220_Alfa_generate_integrated
21    200220_Alfa_generate_integrated(3)
22    200220_Alfa_generate_integrated(5)
23            200221_Alfa_

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [110]:
df_sort = df.copy()
Animal_strs = ["Alfa", "Beto"]
df_paths = [r"S:\Exp_Record_Alfa.xlsx", r"S:\ExpSpecTable_Augment.xlsx"]
if isinstance(df_sort,str):
    df_sort = pd.read_excel(df_sort)
# loop through animal name and sort corresponding exp to the collection
for animal, out_path in zip(Animal_strs, df_paths):
    print("Sort out exp for %s, adding "%animal)
    df_old = pd.read_excel(out_path) # load the old exp collection
    id_col = []
    for idx in df_sort.index:
        name = df_sort.expControlFN[idx]
        if name is np.nan:
            print("%s Empty bhv file entry encountered" % df_sort.ephysFN[idx])
            continue
        if animal.lower() in name.lower():
            if (df_old.expControlFN==name).any():
                print("%s  has been recorded in the excel index %d, please check. Skipping."%(name, (df_old.expControlFN==name).nonzero()[0][0]))
            else:
                id_col.append(idx)
    if len(id_col) == 0:
        print("\nNo new experiments to add! Continue.")
        continue
    df_ftr = df_sort.iloc[id_col].copy()
    print(df_ftr.expControlFN)
    if addexplabel is not None:
        df_ftr.Exp_collection[:] = addexplabel
    df_cat = pd.concat([df_old, df_ftr], axis=0, ignore_index=True)
    df_cat.to_excel(out_path,index=False) # write to the excel of all old experiments 

Sort out exp for Alfa, adding 
Alfa64chan-17022020-003 Empty bhv file entry encountered
0        200204_ALfa_generate_integrated
1     200204_ALfa_generate_integrated(1)
2     200205_ALfa_generate_integrated(2)
3     200205_ALfa_generate_integrated(4)
4        200210_Alfa_generate_integrated
5        200211_Alfa_generate_integrated
6          200213_Alfa_rfMapper_basic(1)
7     200213_Alfa_generate_integrated(1)
8     200213_Alfa_generate_integrated(2)
9             200217_Alfa_rfMapper_basic
10         200217_Alfa_rfMapper_basic(1)
12    200217_Alfa_generate_integrated(4)
13            200218_Alfa_rfMapper_basic
14         200218_Alfa_rfMapper_basic(1)
15       200218_Alfa_generate_integrated
16            200219_Alfa_rfMapper_basic
17         200219_Alfa_rfMapper_basic(1)
18    200219_Alfa_generate_integrated(2)
19            200220_Alfa_rfMapper_basic
20         200220_Alfa_rfMapper_basic(1)
21       200220_Alfa_generate_integrated
22    200220_Alfa_generate_integrated(3)
23    2002

NameError: name 'addexplabel' is not defined

In [108]:
df_sort

,ephysFN,expControlFN,stimuli,comments,Exp_collection,Expi,pref_chan,stim_size
0,Alfa64chan-04022020-002,200204_ALfa_generate_integrated,Stimuli\2019-12-Evolutions\2020-02-04-Alfa-01\...,"Start: 1:38 pm, generate_integrated, parallel ...",NaN,NaN,NaN,NaN
1,Alfa64chan-04022020-003,200204_ALfa_generate_integrated(1),Stimuli\2019-12-Evolutions\2020-02-04-Alfa-02\...,"Start: 2:03 pm, restarted evolution\ngenerate_...",NaN,NaN,NaN,NaN
2,Alfa64chan-05022020-005,200205_ALfa_generate_integrated(2),Stimuli\2019-12-Evolutions\2020-02-05-Alfa-01\...,005 generate_integrated SU vs Hash\nCh. 33 [...,NaN,NaN,NaN,NaN
3,Alfa64chan-05022020-007,200205_ALfa_generate_integrated(4),Stimuli\2019-12-Evolutions\2020-02-05-Alfa-03\...,007 generate_integrated SU vs Hash\nCh. 38 [0...,NaN,NaN,NaN,NaN
4,Alfa64chan-10022020-004,200210_Alfa_generate_integrated,Stimuli\2019-12-Evolutions\2020-02-10-Alfa-01\...,Ch. 7 [0 -2] 3 1 'CMAES'\nCh. 7 [0 -2] 3 2 'CM...,NaN,NaN,NaN,NaN
5,Alfa64chan-11022020-003,200211_Alfa_generate_integrated,Stimuli\2019-12-Evolutions\2020-02-11-Alfa-01\...,"SU v Hash ch. 63 [0, -2] SU 1/5 very visually ...",NaN,NaN,NaN,NaN
6,Alfa64chan-13022020-001,200213_Alfa_rfMapper_basic(1),N:\Stimuli\2019-06-RF-mapping\2020-02-13-Alfa,001RFMapping \nstarts 11:59 AM\n-8:2:8 2deg im...,NaN,NaN,NaN,NaN
7,Alfa64chan-13022020-002,200213_Alfa_generate_integrated(1),NaN,"002 generate_integrated\n63 [0, -2] 3 1 CMAES...",NaN,NaN,NaN,NaN
8,Alfa64chan-13022020-003,200213_Alfa_generate_integrated(2),Stimuli\2019-12-Evolutions\2020-02-13-Alfa-01\...,"003 generate_integrated\nRe open ML2, change t...",NaN,NaN,NaN,NaN
9,Alfa64chan-17022020-001,200217_Alfa_rfMapper_basic,NaN,001 RFMapper starts\n-1.5:0.5:1.5 mapping 1 deg,NaN,NaN,NaN,NaN


## Matlab Data Processing

In [5]:
format compact

In [1]:
Set_Path;

Drive already SUBSTed 

ans =

     1




In [4]:
winopen("S:\ExpSpecTable_Augment.xlsx")

In [2]:
open("Project_Manifold_Beto_loadRaw")

In [3]:
open("Evol_Optimizer_Cmp")